### Construct Historical Data
- quandl.com 에서 과거데이터를 다운받아 저장함

In [1]:
import csv
import h5py
import quandl
import numpy as np
import pandas as pd

In [2]:
quandl.ApiConfig.api_key = "6GRDzQGAm5PpBzWdcBT5"
METAFILE = 'instruments.csv'
DBFILE = 'history/futures.h5py'

#### 1. 상품정보 불러오기 instruments.csv -> dictionary

In [3]:
with open(METAFILE ,'r') as file:
    reader = csv.DictReader(open('instruments.csv'))
    instruments = []
    for row in reader:
        instruments.append(row)

#### 2. 데이터 다운받아 h5py로 저장

In [4]:
db = h5py.File(DBFILE, mode='w')

In [5]:
db.attrs['columns'] = 'date;open;high;low;close;volume;op_int'

In [6]:
for instrument in instruments[:1]:
    if instrument['has_history'] == 'y':
        print(instrument['name'])
        exch = instrument['exchange'].lower()
        symbol = instrument['symbol'].lower()
        depth = instrument['depth']
        code = f"{exch}_{symbol}{depth}_OB"
        df=quandl.get_table('SCF/PRICES', quandl_code=code, paginate=True)[::-1]
        df['date'] = df['date'].astype('int64')/1e6
        data = df[['date','open','high','low','settle','volume','prev_day_open_interest']]
        group = db.create_dataset((exch+'_'+symbol).lower(), data.shape, data=data, dtype='float64')
        group.attrs['symbol'] = symbol
        group.attrs['exchange'] = exch
        group.attrs['name'] = instrument['name'].lower()
        group.attrs['code'] = code.lower()

Australian Dollar


In [7]:
db.close()

#### 3. 확인

In [8]:
db = h5py.File(DBFILE, mode='r')

In [9]:
d=db['CME_AD']

KeyError: "Unable to open object (object 'CME_AD' doesn't exist)"

In [12]:
columns = db.attrs['columns'].split(';')

In [13]:
df = pd.DataFrame(d[:], columns=columns)

In [14]:
df['date'] = df['date'].astype('M8[s]')

In [15]:
df

,date,open,high,low,close,volume,op_int
0,1960-10-26,24.31,24.32,24.31,24.31,0.0,0.0
1,1960-10-27,24.28,24.31,24.28,24.28,1.0,1.0
2,1960-10-28,24.32,24.32,24.23,24.32,27.0,27.0
3,1960-10-31,24.43,24.51,24.37,24.43,12.0,34.0
4,1960-11-01,24.37,24.45,24.37,24.37,24.0,52.0
...,...,...,...,...,...,...,...
14902,2020-01-13,34.45,34.55,33.90,33.95,54160.0,247872.0
14903,2020-01-14,33.98,34.15,33.51,34.07,65784.0,246803.0
14904,2020-01-15,34.08,34.11,33.25,33.30,82343.0,241919.0
14905,2020-01-16,33.30,33.45,32.89,33.03,67631.0,238702.0


In [16]:
db.close()